Copyright © 2020, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved.
SPDX-License-Identifier: Apache-2.0

# Administer User Groups

This example shows how to administer user group identities for the following tasks:

* Get a list of custom groups
* Display a list of the group members
* Create a new custom group
* Add members to a group


### Step 1: Initialization

In [ ]:
import requests
import json
import getpass
import base64

host = "http://myserver.com"

### Step 2: Get Authoritzation Token

In [ ]:
authURI = "/SASLogon/oauth/token"

headersAuth = {
    'accept': "application/json",
    'content-type': "application/x-www-form-urlencoded",
    'Authorization': "Basic c2FzLmVjOg=="}
authToken = ''
username = ''
password = ''
notAuthenticated = True

while notAuthenticated:
    username = input("Enter adminstrator username: ")
    password = getpass.getpass(f"Enter password for user {username}: ")
    authBody = f"grant_type=password&username={username}&password={password}"
    authPost = requests.post(host + authURI,
                             data=authBody,
                             headers=headersAuth)
    if authPost.status_code == requests.codes.ok:
        authToken = authPost.json()['access_token']
        notAuthenticated = False
    else:
        print("Please enter a valid username and password combination.")

password = ''

### Step 3: Update User Groups

In [ ]:
selectedIndex = 0
addMembers = True
selectedGroup = {}
selectedIndex = 0
index = 0
groupName = ""

import sys
from io import StringIO
from IPython import get_ipython


class IpyExit(SystemExit):
    """Exit Exception for IPython.

    Exception temporarily redirects stderr to buffer.
    """
    def __init__(self):
        # print("exiting")  # optionally print some message to stdout, too
        # ... or do other stuff before exit
        sys.stderr = StringIO()

    def __del__(self):
        sys.stderr.close()
        sys.stderr = sys.__stderr__  # restore from backup


def ipy_exit():
    raise IpyExit


if get_ipython():    # ...run with IPython
    exit = ipy_exit  # rebind to custom exit
else:
    exit = exit      # just make exit importable
    
def getCustomGroups():
    headersGetGroups={'Authorization': 'Bearer ' + authToken,
                  'Accept': 'application/json'}
    identities = requests.get(host + "/identities/groups?providerId=local&start=0&limit=100&sortBy=name",
                                    headers = headersGetGroups)
    if identities.status_code == 200:
        print('Log in successful')
    else:
        print('Log in failed')
        raise IpyExit
        
    return identities.json()['items']
    
def displayGroupNames(customGroups):
    index = 1
    for group in customGroups:
        print(str(index) + ". " + group['name'])
        index = index + 1
    print(str(index) + ". Create New Group")
    selectedIndex = input("Select group to edit:")
    return index, selectedIndex

def getSelectedGroup():
    newGroup = False
    try:
        int(selectedIndex)
    except ValueError:
        print('Invalid entry. Exiting.')
        raise IpyExit
        
    if int(selectedIndex) == index:
        addMembers = True
        groupName = input("Enter new group name:")
        groupId = input("Enter new group ID:")
        selectedGroup = {'name': groupName, 'id': groupId}
        newGroup = True
    elif int(selectedIndex) > 0 and int(selectedIndex) < index:
        selectedGroup = customGroups[int(selectedIndex) -1] 
        groupName = selectedGroup['name']
        groupId = selectedGroup['id']
        print("1. List group members")
        print("2. Add new group members")
        selectedOpt = input("Select: ")
        if int(selectedOpt) == 1:
            addMembers = False
        elif int(selectedOpt) == 2:
            addMembers = True
        else:
            getSelectedGroup()
    else:
        print("Selected item not known")
        addMembers = False
        groupName = ""
        groupId = ""
        raise IpyExit
        
    return addMembers, groupName, groupId, newGroup

def addGroupMembers(groupId, groupName, newGroup):
    memberName = ""
    memberList = []
    if newGroup:
        headersUpdateGroup={'Authorization': 'Bearer ' + authToken,
                          'Content-type': 'application/json'}
        body = {"id": groupId, "name": groupName}
        identities = requests.post(host + "/identities/groups/",
                                    headers = headersUpdateGroup,
                                    data = json.dumps(body))
        if identities.status_code != 201:
            print(groupName + ' was created successfully')
        else:
            print('Group: ' + groupName + ' could not be created with the status: ' + str(identities.status_code))

    while memberName == "":
        memberName = input("Enter member user ID <leave blank when done>:")
        if memberName != "":
            memberList.append(memberName)
            memberName = ""
        else:
            break
    for userName in memberList:
        headersUpdateGroup={'Authorization': 'Bearer ' + authToken,
                          'Content-type': 'application/json'}
        identities = requests.put(host + "/identities/groups/" + groupId + "/userMembers/" + userName,
                                    headers = headersUpdateGroup)
        if identities.status_code != 201:
            print("User: " + userName + " could not be added to the group with the status: " + str(identities.status_code))
        else:
            print("User: " + userName + " was added successfully.")

def listGroupMembers(groupName):
        headersGetMembers={'Authorization': 'Bearer ' + authToken,
                          'Accept': 'application/json'}
        identitiesResponse = requests.get(host + "/identities/groups/" + groupName + "/members",
                                    headers = headersGetMembers)
        userMembers = []
        identities = identitiesResponse.json()
        if identities['count'] > 0:
            userMembers = identities['items']
            for userMember in userMembers:
                print("ID: " + userMember['id'] + "  Name: " + userMember['name'])
        else:
            print("No members for this group.")

customGroups = getCustomGroups()
groupSelections = displayGroupNames(customGroups)
index = groupSelections[0]
selectedIndex = groupSelections[1]
groupResult = getSelectedGroup()
addMembers = groupResult[0]
groupName = groupResult[1]
groupId = groupResult[2]
newGroup = groupResult[3]

if addMembers:
    addGroupMembers(groupId, groupName, newGroup)
else:
    listGroupMembers(groupId)